<a href="https://colab.research.google.com/github/bhartiansh/cnn_pruning_cifar10/blob/main/l2_Covnet_pruned_resnet20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bhartiansh/cnn_pruning_cifar10.git
%cd cnn_pruning_cifar10

!pip install -q tensorflow-model-optimization

import tensorflow as tf
import numpy as np
from models.resnet20 import build_resnet20
from tensorflow.keras.datasets import cifar10

Cloning into 'cnn_pruning_cifar10'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 202 (delta 29), reused 2 (delta 2), pack-reused 144 (from 1)
Receiving objects: 100% (202/202), 117.14 KiB | 895.00 KiB/s, done.
Resolving deltas: 100% (92/92), done.
/content/cnn_pruning_cifar10
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 29.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
def get_conv_layers(model):
    return [layer for layer in model.layers if isinstance(layer, tf.keras.layers.Conv2D)]

def compute_filter_saliency(model):
    saliency = {}
    for layer in get_conv_layers(model):
        weights = layer.get_weights()[0]
        l2_norms = np.linalg.norm(weights.reshape(-1, weights.shape[-1]), axis=0)
        saliency[layer.name] = l2_norms
    return saliency

def prune_filters(model, saliency, sparsity):
    for layer in get_conv_layers(model):
        weights = layer.get_weights()
        if not weights:
            continue
        kernel, *rest = weights
        num_filters = kernel.shape[-1]
        num_prune = int(sparsity * num_filters)
        if num_prune == 0:
            continue

        l2_norms = saliency[layer.name]
        prune_indices = np.argsort(l2_norms)[:num_prune]
        kernel[..., prune_indices] = 0
        if rest:
            bias = rest[0]
            bias[prune_indices] = 0
            layer.set_weights([kernel, bias])
        else:
            layer.set_weights([kernel])

def train_l2_pruned_resnet20(x_train, y_train, x_val, y_val,
                              sparsity=0.3, pre_epochs=2, fine_tune_epochs=30, batch_size=64):
    model = build_resnet20()

    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(50000).batch(batch_size).prefetch(AUTOTUNE)
    val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size).prefetch(AUTOTUNE)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_ds, epochs=pre_epochs, validation_data=val_ds, verbose=2)

    saliency = compute_filter_saliency(model)
    prune_filters(model, saliency, sparsity=sparsity)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(train_ds, epochs=fine_tune_epochs, validation_data=val_ds, verbose=2)

    return model, history

In [5]:
# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Run the training
model, history = train_l2_pruned_resnet20(x_train, y_train, x_test, y_test, sparsity=0.3)

Epoch 1/2
782/782 - 60s - loss: 1.3278 - accuracy: 0.5168 - val_loss: 1.2876 - val_accuracy: 0.5637 - 60s/epoch - 77ms/step
Epoch 2/2
782/782 - 54s - loss: 0.9217 - accuracy: 0.6726 - val_loss: 1.0007 - val_accuracy: 0.6552 - 54s/epoch - 69ms/step
Epoch 1/30
782/782 - 60s - loss: 0.8181 - accuracy: 0.7119 - val_loss: 0.9076 - val_accuracy: 0.6927 - 60s/epoch - 77ms/step
Epoch 2/30
782/782 - 55s - loss: 0.6654 - accuracy: 0.7698 - val_loss: 0.7272 - val_accuracy: 0.7461 - 55s/epoch - 70ms/step
Epoch 3/30
782/782 - 54s - loss: 0.5871 - accuracy: 0.7963 - val_loss: 0.7485 - val_accuracy: 0.7414 - 54s/epoch - 70ms/step
Epoch 4/30
782/782 - 54s - loss: 0.5274 - accuracy: 0.8187 - val_loss: 0.7621 - val_accuracy: 0.7420 - 54s/epoch - 69ms/step
Epoch 5/30
782/782 - 55s - loss: 0.4779 - accuracy: 0.8359 - val_loss: 0.6856 - val_accuracy: 0.7693 - 55s/epoch - 70ms/step
Epoch 6/30
782/782 - 54s - loss: 0.4291 - accuracy: 0.8533 - val_loss: 0.7062 - val_accuracy: 0.7654 - 54s/epoch - 69ms/step
Ep

In [ ]:
# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Run the training
model, history = train_l2_pruned_resnet20(x_train, y_train, x_test, y_test, sparsity=0.5)

Epoch 1/2
782/782 - 61s - loss: 1.3282 - accuracy: 0.5156 - val_loss: 1.1924 - val_accuracy: 0.5807 - 61s/epoch - 79ms/step
Epoch 2/2
782/782 - 55s - loss: 0.9024 - accuracy: 0.6813 - val_loss: 0.9990 - val_accuracy: 0.6598 - 55s/epoch - 70ms/step
Epoch 1/30
782/782 - 61s - loss: 1.0168 - accuracy: 0.6417 - val_loss: 1.0733 - val_accuracy: 0.6385 - 61s/epoch - 78ms/step
Epoch 2/30
782/782 - 55s - loss: 0.7614 - accuracy: 0.7353 - val_loss: 0.8404 - val_accuracy: 0.7080 - 55s/epoch - 71ms/step
Epoch 3/30
782/782 - 54s - loss: 0.6689 - accuracy: 0.7687 - val_loss: 0.8885 - val_accuracy: 0.7106 - 54s/epoch - 69ms/step
Epoch 4/30
782/782 - 54s - loss: 0.6107 - accuracy: 0.7887 - val_loss: 0.9136 - val_accuracy: 0.7004 - 54s/epoch - 69ms/step
Epoch 5/30
782/782 - 54s - loss: 0.5571 - accuracy: 0.8057 - val_loss: 0.7812 - val_accuracy: 0.7413 - 54s/epoch - 69ms/step
Epoch 6/30
782/782 - 54s - loss: 0.5153 - accuracy: 0.8202 - val_loss: 0.7593 - val_accuracy: 0.7483 - 54s/epoch - 69ms/step
Ep

In [ ]:
# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Run the training
model, history = train_l2_pruned_resnet20(x_train, y_train, x_test, y_test, sparsity=0.7)

In [ ]:
# 1. Force compatible versions (reset everything to Colab defaults)
!pip install -U --force-reinstall numpy==1.23.5
!pip install -U --force-reinstall tensorflow==2.14.0
!pip install -U tensorflow-model-optimization

# 2. Restart runtime automatically after install
import os
os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 89.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 226.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 105.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Attempting uninstall: absl-py
    Found existing installation: absl-py 2.2.2
    Uninstalling absl-py-2.2.2:
      Successfully uninstalled absl-py-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires ml_dtypes>=0.4.0, but you have ml-dtypes 0.2.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.14.0 which is incompatible.